In [163]:
import numpy as np
import pandas as pd

$\newcommand{\diag}[1]{\text{diag}\left(#1\right)}$
# US Exercise

We are going to use the data calibrated to the US economy to evaluate the impact of technology and labor supply shocks. 

In [164]:
data_dir = '../data/clean/'
dfA      = pd.read_csv(data_dir + 'A.csv')
dfParam  = pd.read_csv(data_dir + 'params.csv')
dfLshare = pd.read_csv(data_dir + 'labor_share.csv')
dfLabor_market_yearly= pd.read_csv(data_dir + 'labor_market_yearly.csv')
dfLabor_market_yearly = dfLabor_market_yearly.sort_values(by=['year', 'BEA_sector'])
dfLabor_market_yearly = dfLabor_market_yearly.dropna(axis=0)
dfLabor_market_yearly = dfLabor_market_yearly[dfLabor_market_yearly['year'] == 2005]
# reformatting parameters
Omega = np.array(dfA.iloc[:, 1:], dtype='float64')
Psi = np.linalg.inv(np.eye(Omega.shape[0])-Omega)
J = Omega.shape[0]

φ = np.array(dfParam.φ)

epsN = np.diag(np.array(dfParam.α))
epsD = np.array(dfParam.θ_alt)

We need to convert the paramters from the data to the right format for our code. We need the input matrix $\Omega$, diagonal matrices of matching elasticities and the production elasticity to labor. In addition our data contains separate matching efficiencies for each sector. We need to encorporate these into the matrices $\diag{\bm{\mathcal{Q}}}$, $\diag{\bm{\mathcal{F}}}$, and $\diag{\bm{\tau}}$.

In [165]:
θ = dfLabor_market_yearly['v'] / dfLabor_market_yearly['u'] 
ν = np.ones(J) * 0.5
curlyQ = np.diag(-ν)
curlyF =  np.eye(J) + curlyQ
r = 0.045
tau = np.diag(r / (φ*θ**(-ν) - r))
# NOTE: here I am picking the r so the τ's roughly look right. Down the line, we can calibrate τ directly from the data. 
# Also note that tightness is computed from unemployed workers, but in our model it should be computed from the whole work force.
tau

array([[0.02299856, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.        , 0.01846423, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.01315401, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.03653684, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.      

## Defining shocks
We are interested in the response of sector level and aggregate output and employment to technology shocks $d\log\bm{A}$ and labor force shocks $d\log \bm{H}$. The code below defines the shocks we feed into the model.

In [166]:
# Technology shocks
dlog_A = 0.01*np.zeros(J)
dlog_A[0] = 0.01
dlog_H = 0.01*np.zeros(J)

## Defining how wages adjust
Since there are mutual gains from trade once an unemployed worker and a firm meet, wages are not pinned down uniquely in models featuring search and matching frictions in the labor market. We must therefore impose a wage schedule, an assumption about how wages change in response to fundamentals, in order to close the model. 

Let $W_i$ be the nominal sector wage and $w_i = W_i / P_i$ the wage for sector i adjusted for goods price in that sector. 

In particular, we need to specify the elasticity of wages to technology shocks and labor force shocks in each sector, $\left\{\left\{\varepsilon^{w_j}_{A_{ji}},\varepsilon^{w_j}_{H_{ji}}\right\}_{i=1}^{J}\right\}_{j=1}^{J}$. For instance, a simple, but unrealistic, assumption is that wages respond positively to own sector productivity and negatively to own sector labor force, but do not respond to changes in other sectors. In general, we express changes in wages as a function of changes in productivity and the labor force. 
\begin{align}
    d\log \bm{w} &= \bm{\Lambda_{A}} d\log \bm{A} + \bm{\Lambda_{H}} d\log \bm{H} \tag{1}
\end{align}
Where $\bm{\Lambda_{A}}$ contains wage elasticities to productivity changes and $\bm{\Lambda_{H}}$ contains wage elasticities to labor force changes.

In [167]:
# Wage elasticities
# Start with fixed real wages
epsW_A = 0.01 * np.ones_like(Psi) + 0.99*np.eye(Psi.shape[0])
epsW_H = 0.01 * np.ones_like(Psi) + 0.99*np.eye(Psi.shape[0])
epsW_A

array([[1.  , 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
        0.01, 0.01, 0.01, 0.01, 0.01, 0.01],
       [0.01, 1.  , 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
        0.01, 0.01, 0.01, 0.01, 0.01, 0.01],
       [0.01, 0.01, 1.  , 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
        0.01, 0.01, 0.01, 0.01, 0.01, 0.01],
       [0.01, 0.01, 0.01, 1.  , 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
        0.01, 0.01, 0.01, 0.01, 0.01, 0.01],
       [0.01, 0.01, 0.01, 0.01, 1.  , 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
        0.01, 0.01, 0.01, 0.01, 0.01, 0.01],
       [0.01, 0.01, 0.01, 0.01, 0.01, 1.  , 0.01, 0.01, 0.01, 0.01, 0.01,
        0.01, 0.01, 0.01, 0.01, 0.01, 0.01],
       [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 1.  , 0.01, 0.01, 0.01, 0.01,
        0.01, 0.01, 0.01, 0.01, 0.01, 0.01],
       [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 1.  , 0.01, 0.01, 0.01,
        0.01, 0.01, 0.01, 0.01, 0.01, 0.01],
       [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01

In [168]:
# Calculating the log change in wages
def WageFunc(dlog_A, dlog_H, epsW_A, epsW_H):
    dlog_wR = epsW_A @ dlog_A + epsW_H @ dlog_H
    return dlog_wR

In [169]:
#How wages change
dlog_wR = WageFunc(dlog_A, dlog_H, epsW_A, epsW_H)
dlog_wR

array([0.01  , 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001,
       0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001,
       0.0001])


## Tightness propagation
We with wage changes in hand, we solve for first order changes in tightness in terms of $d\log\bm{A}$, $d\log\bm{H}$, and $d\log\bm{w}$. The general formula for changes in tightness, treating sector 1 prices as the numeraire, is
\begin{align}
    d \log \bm{\theta} &=\left(\diag{\bm{\mathcal{F}}}-\bm{\Xi_{\theta}}\right)^{-1}\left(\left(\bm{I} - \bm{\Psi} \diag{\bm{\varepsilon^f_N}}\right) \left(d\log \bm{\varepsilon^f_N} + d\log \bm{\lambda} - d\log \bm{H}\right) + \bm{\Psi} d\log \bm{A}\right) \\
    &- \left(\diag{\bm{\mathcal{F}}}-\bm{\Xi_{\theta}}\right)^{-1} \left(d\log \bm{w}-d\log\bm{p}\right) \tag{2}
\end{align}
Where 
\begin{align*}
    \bm{\Xi_\theta} = \bm{\Psi}\diag{\bm{\varepsilon^f_N}}\left(\diag{\bm{\mathcal{F}}} + \diag{\bm{\tau}}\diag{\bm{\varepsilon^{\mathcal{Q}}_{\theta}}}\right)
\end{align*}

In [170]:
def ThetaFunc(dlog_A, dlog_H, dlog_wR, dlog_epsN, dlog_lam, Psi, curlyF, curlyQ, epsN, tau):
    
    # Creating matrices
    Xi_a = Psi

    Xi_theta = Psi @ epsN @ curlyF+ Psi @ epsN @ tau @ curlyQ
    #Xi_theta[num,:] = Xi_theta[num,:] + Psi[num,:] @ epsN @ tau @ curlyQ

    Xi_w = np.zeros_like(Psi)

    I = np.eye(Psi.shape[0])

    # Contribution of different components
    Cw = np.linalg.inv(curlyF - Xi_theta) @ (I - Xi_w)
    Ca = np.linalg.inv(curlyF - Xi_theta) @ Xi_a 
    Ch = np.linalg.inv(curlyF - Xi_theta) @ (I - Psi @ epsN)

    # Change in tightness
    dlog_theta = Ch @ (dlog_epsN + dlog_lam - dlog_H) + Ca @ dlog_A - Cw @ dlog_wR

    return dlog_theta

Assuming Cobb-Douglas production implies $d\log\bm{\varepsilon^f_N} = d\log \bm{\lambda} = 0$.

In [171]:
dlog_epsN = np.zeros_like(dlog_A)
dlog_lam = np.zeros_like(dlog_A)
dlog_theta = ThetaFunc(dlog_A, dlog_H, dlog_wR, dlog_epsN, dlog_lam, Psi, curlyF, curlyQ, epsN, tau)
dlog_theta

array([0.01058486, 0.01033268, 0.00957985, 0.00962716, 0.00980302,
       0.00994056, 0.00929646, 0.01000874, 0.00991005, 0.00980922,
       0.00983027, 0.0097472 , 0.01027403, 0.01067027, 0.00989899,
       0.01008175, 0.00971456])

## Price and output propagation
With changes in tightness in hand, we can now work out how prices and sectoral production changes in response to technology and labor force shocks. Price changes are given by 
\begin{align}
    \left(I - \Psi\text{diag}\left(\bm{\varepsilon^{f}_{N}}\right)\right)d \log \bm{p} &=\bm{\Psi} \left(
        \text{diag}\left(\bm{\varepsilon^{f}_{N}}\right) \left(d\log\bm{w}  - d\log\bm{p}\right) -\text{diag}\left(\bm{\varepsilon^{f}_{N}}\right)\text{diag}\left(\bm{\tau}\right)\text{diag}\left(\bm{\varepsilon^{\mathcal{Q}}_{\theta}}\right)d\log\bm{\theta} - d\log \bm{A} \right) \tag{3}
\end{align}
While $\Xi = \left(I - \Psi\text{diag}\left(\bm{\varepsilon^{f}_{N}}\right)\right)$ is not invertible, we impose an additional restrction $d\log p_i = 0$, with the good produced by sector $i$ the numeraire good of our choice.

In [172]:
def PriceFunc(dlog_A, dlog_wR, dlog_theta, Psi, curlyQ, epsN, tau, num=0):
    # Contributions of different components
    Cw = Psi @ epsN 
    Cw[num, :] = 0
    Ct = Psi @ epsN @ tau @ curlyQ
    Ct[num, :] = 0
    Ca = np.copy(Psi)
    Ca[num, :] = 0
    
    Xi = np.eye(Psi.shape[0]) - Psi @ epsN 
    Xi[num, :] = 0
    Xi[num, num] = 1

    # Price changes
    dlog_p = np.linalg.inv(Xi) @ (Cw @ dlog_wR - Ct @ dlog_theta - Ca @dlog_A)
    return dlog_p


In [173]:
num = 0
dlog_p = PriceFunc(dlog_A, dlog_wR, dlog_theta, Psi, curlyQ, epsN, tau, num)
dlog_p

array([0.        , 0.01002609, 0.01040251, 0.01037885, 0.01029092,
       0.01022215, 0.0105442 , 0.01018806, 0.0102374 , 0.01028782,
       0.01027729, 0.01031883, 0.01005542, 0.00985729, 0.01024294,
       0.01015155, 0.01033515])

Alternatively, we can solve explicitly for $d\log\bm{p}$ 
\begin{align*}
    d\log \bm{p} &= \bm{\Xi_p}^{-1}\text{diag}\left(\bm{\varepsilon^{f}_{N}}\right) \left(d\log\bm{w} - d\log \bm{p}\right)\nonumber \\
    &-\bm{\Xi_p}^{-1}\text{diag}\left(\bm{\varepsilon^{f}_{N}}\right)\text{diag}\left(\bm{\tau}\right)\text{diag}\left(\bm{\varepsilon^{\mathcal{Q}}_{\theta}}\right)d\log\bm{\theta}\\
    &- \bm{\Xi_p}^{-1} d\log \bm{A}
\end{align*}
Where
\begin{align*}
    \bm{\Xi_{p}} &= \begin{bmatrix}
        0 & \bm{0}_{1\times (J-1)} \\
        \bm{0}_{(J-1)\times 1} & \bm{I}_{(J-1) \times (J-1)} - \diag{\bm{\varepsilon^f_N}\setminus\varepsilon^{f_1}_{N_1}}
    \end{bmatrix} - \bm{\Omega}
\end{align*}
The two solution methods should coincide to machine precision. 

In [174]:
def PriceFunc2(dlog_A, dlog_wR, dlog_theta, Omega, curlyQ, epsN, tau, num = 0):
    Xi_p = np.eye(Omega.shape[0]) - epsN 
    Xi_p[num, : ] = 0 
    Xi_p = Xi_p - Omega
    dlog_p = np.linalg.inv(Xi_p) @ (epsN @ (dlog_wR - tau @ curlyQ @ dlog_theta) - dlog_A)
    return dlog_p

In [175]:
dlog_p2 = PriceFunc2(dlog_A, dlog_wR, dlog_theta, Omega, curlyQ, epsN, tau, num)
dlog_p2


array([8.69530142e-16, 1.00260894e-02, 1.04025058e-02, 1.03788498e-02,
       1.02909201e-02, 1.02221506e-02, 1.05441974e-02, 1.01880593e-02,
       1.02374030e-02, 1.02878187e-02, 1.02772939e-02, 1.03188295e-02,
       1.00554153e-02, 9.85729386e-03, 1.02429358e-02, 1.01515550e-02,
       1.03351499e-02])

We can check that our solution is consistent with $p_i$ being the numeraire by checking that 
\begin{align*}
    0 & = \varepsilon^{f_i}_{N_i} \left( d\log w_i - d\log p_i - \tau_i(\theta_i) \varepsilon^{\mathcal{Q_i}}_{\theta_i} d\log \theta_i\right)+\Omega_i d\log\bm{p} - d\log A_i 
\end{align*}

In [176]:
epsN[num,num] * (dlog_wR[num] - tau[num,num] * curlyQ[num,num] * dlog_theta[num]) + Omega[num,] @ dlog_p - dlog_A[num]

-1.0269562977782698e-15

In [177]:
epsN[num,num] * (dlog_wR[num] - tau[num,num] * curlyQ[num,num] * dlog_theta[num]) + Omega[num,] @ dlog_p2 - dlog_A[num]

1.734723475976807e-18

Finally, we can check that our two solution methods coincide, providing another sanity check on our code.

In [178]:
dlog_p - dlog_p2

array([-8.69530142e-16, -1.92033889e-15, -1.92727778e-15, -1.92033889e-15,
       -1.92554306e-15, -1.92033889e-15, -1.91166527e-15, -1.92207361e-15,
       -1.90819582e-15, -1.92033889e-15, -1.92033889e-15, -1.92380833e-15,
       -1.92207361e-15, -1.92207361e-15, -1.91686944e-15, -1.91686944e-15,
       -1.91860416e-15])

And output changes are given by
\begin{align}
    d\log \bm{y} &= \bm{\Psi}\left(d\log\bm{A} + \text{diag}\left(\bm{\varepsilon^{f}_{N}}\right)\left(\text{diag}\left(\bm{\mathcal{F}}\right)+\text{diag}\left(\bm{\tau}\right)\text{diag}\left(\bm{\varepsilon^{\mathcal{Q}}_{\theta}}\right)\right)d\log \bm{\theta} + \text{diag}\left(\bm{\varepsilon^{f}_{N}}\right) d\log\bm{H}\right) \nonumber\\
    &-\bm{\Psi} \text{diag}\left(\bm{\varepsilon^{f}_{N}}\right) d\log \bm{\varepsilon^{f}_{N}} + \left(\bm{I} - \bm{\Psi}\text{diag}\left(\bm{\varepsilon^{f}_{N}}\right)\right) d\log \bm{\lambda} \tag{4}
\end{align}


In [179]:
def OutputFunc(dlog_A, dlog_H, dlog_theta, dlog_epsN, dlog_lam, Psi, curlyQ, curlyF, epsN, tau):
    # Contributions of different coponents
    Ca = Psi
    Ct = Psi @ epsN @ (curlyF + tau @ curlyQ) 
    Ch = Psi @ epsN 
    I = np.eye(Psi.shape[0])

    # Changes in output
    dlog_y = Ca @ dlog_A + Ct @ dlog_theta + Ch @ (dlog_H-dlog_epsN) + (I - Ch) @ dlog_lam 

    return dlog_y

In [180]:
dlog_y = OutputFunc(dlog_A, dlog_H, dlog_theta, dlog_epsN, dlog_lam, Psi, curlyQ, curlyF, epsN, tau)
dlog_y

array([0.01529243, 0.00526634, 0.00488992, 0.00491358, 0.00500151,
       0.00507028, 0.00474823, 0.00510437, 0.00505503, 0.00500461,
       0.00501514, 0.0049736 , 0.00523701, 0.00543514, 0.00504949,
       0.00514087, 0.00495728])

With Cobb-Douglas production, real output should change by the same amount in each sector. This provides an overall check of our code given the Cobb-Douglas assumption.

In [181]:
# Change in nominal GDP
dlog_p + dlog_y

array([0.01529243, 0.01529243, 0.01529243, 0.01529243, 0.01529243,
       0.01529243, 0.01529243, 0.01529243, 0.01529243, 0.01529243,
       0.01529243, 0.01529243, 0.01529243, 0.01529243, 0.01529243,
       0.01529243, 0.01529243])

We can now check consistency. We should get the same change in labor by either calculating the change in labor supply
\begin{align}
    d\log \bm{L^s} = \text{diag}\left(\mathcal{F}\right) d\log \bm{\theta} + d\log \bm{H} \tag{5}
\end{align}
Or the change in labor demand
\begin{align}
    d\log \bm{L^d} = d\log \bm{\varepsilon^{f}_N} + d\log \bm{p} + d\log\bm{y} - d\log\bm{w} \tag{6}
\end{align}

In [182]:
def LaborSupply(dlog_H,dlog_theta,curlyF):
    dlog_Ls = curlyF @ dlog_theta + dlog_H
    return dlog_Ls
    
def LaborDemand(dlog_wR, dlog_y, dlog_epsN):
    dlog_Ld = dlog_epsN  + dlog_y - dlog_wR
    return dlog_Ld

Consistency requires
\begin{align*}
    d\log \bm{L^d} - d\log\bm{L^s} = 0
\end{align*}

In [183]:
LaborDemand(dlog_wR, dlog_y, dlog_epsN) - LaborSupply(dlog_H,dlog_theta,curlyF)

array([ 1.73472348e-18, -8.67361738e-19, -1.73472348e-18,  0.00000000e+00,
       -8.67361738e-19,  2.60208521e-18, -8.67361738e-19, -1.73472348e-18,
        0.00000000e+00,  0.00000000e+00, -3.46944695e-18,  0.00000000e+00,
       -1.73472348e-18, -4.33680869e-18,  0.00000000e+00,  0.00000000e+00,
       -8.67361738e-19])

## Unemployment
We can use changes in tightness and the labor force to calculate changes in employment. 
\begin{align}
    d\log \bm{L} = \text{diag}\left(\mathcal{F}\right) d\log \bm{\theta} + d\log \bm{H} \tag{5}
\end{align}

In [184]:
dlog_L = LaborSupply(dlog_H,dlog_theta,curlyF)
dlog_L

array([0.00529243, 0.00516634, 0.00478992, 0.00481358, 0.00490151,
       0.00497028, 0.00464823, 0.00500437, 0.00495503, 0.00490461,
       0.00491514, 0.0048736 , 0.00513701, 0.00533514, 0.00494949,
       0.00504087, 0.00485728])

Unemployment in sector $j$ is $U_j = H_j - L_j$. The log change in Unemployment is 
\begin{align}
    d\log \bm{U} = \diag{\bm{U}}^{-1}\left(\diag{\bm{H}} d\log \bm{H} - \diag{L}d\log\bm{L}\right)\tag{6}
\end{align}
The unemployment rate in sector $j$ is $u_j = \frac{H_j-L_j}{H_j}$. The log change in the unemployment rate is 
\begin{align}
    d\log \bm{u} = \left(\bm{I} - \diag{\bm{u}}\right)\diag{\bm{u}}^{-1} \left(d\log\bm{H} - d\log\bm{L}\right) \tag{7}
\end{align}


In [185]:
def UnemploymentFunc(dlog_H, dlog_L, U, H, L):
    dlog_U = np.linalg.inv(U) @ (H @ dlog_H - L @ dlog_L)
    return dlog_U

def UnRateFunc(dlog_H, dlog_L, u):
    dlog_u = (np.eye(dlog_H.shape[0]) - u) @ np.linalg.inv(u) @ (dlog_H - dlog_L)
    return dlog_u

## Aggregation
Given first order changes in output, we can use the elasticities of demand and changes in domar weights calculate changes in aggregate output.
\begin{align}
    d\log Y = \bm{\varepsilon^{\mathcal{D}}_{c}}' \left(d\log\bm{\varepsilon^{\mathcal{D}}_{c}} + d\log \bm{y} - d\log \bm{\lambda}\right) \tag{8}
\end{align}

In [186]:
def AggOutputFunc(dlog_y, dlog_epsD, dlog_lam, epsD):
    dlog_Y = epsD.T @ (dlog_epsD + dlog_y - dlog_lam)
    return dlog_Y

Assuming Cobb-Douglas, $d\log \bm{\varepsilon^{\mathcal{D}}_{c}} = d\log \bm{\lambda} = \bm{0}$. 

In [187]:
dlog_epsD = np.zeros_like(dlog_A)
dlog_Y = AggOutputFunc(dlog_y, dlog_epsD, dlog_lam, epsD)
dlog_Y

0.0055867057842401546